# Web Scraping - Strava

In [1]:
## Biblioteca necessárias

import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    StaleElementReferenceException,
)


## Iniciando uma instância do google chrome

In [2]:
# Pega o caminho do executável do ChromeDriver
service = Service(
    ChromeDriverManager().install()
)  # Corrigido, agora está criando o objeto Service corretamente.

# Usado para definir as preferências para o navegador Chrome
options = webdriver.ChromeOptions()

# Inicia-se uma instância do Google Chrome com as definições
driver = webdriver.Chrome(
    service=service, options=options
)  # Agora a passagem dos parâmetros está correta.

sleep(1)

### Entrando no Site - Strava
url = "https://www.strava.com/segments"
# Segmento lamision35km2023
driver.get(url)

In [4]:
url_2 = "https://www.strava.com/segments/35231066"
driver.get(url_2)

In [4]:
# Login
usuario = "joaomarcos1296@gmail.com"
senha = "Jo@o1296"

## Função tabelas Corredores

### Tabelas dados voltas corredores

In [5]:
def tem_tabela_voltas():
    """
    Verifica se a aba 'Voltas' está disponível e clica nela se existir.
    Retorna True se conseguiu clicar, False caso contrário.
    """
    try:
        # Busca o elemento <li> que contém 'data-tracking-element="laps"'
        elemento_voltas = driver.find_element(
            By.CSS_SELECTOR, 'li[data-tracking-element="laps"]'
        )
        # Se encontrou o elemento, clica nele
        elemento_voltas.click()
        sleep(2)
        return True
    except:
        return False

In [6]:
def pega_tabela_voltas():
    """
    Extrai a tabela de voltas da página de atividades do Strava e salva como CSV.
    """

    # Encontrar a tabela
    table = driver.find_element(By.XPATH, '//*[@id="efforts-table"]')

    # Pegar os cabeçalhos
    headers = [th.text for th in table.find_elements(By.TAG_NAME, "th")]

    # Pegar os dados da tabela
    data = []
    for row in table.find_elements(By.TAG_NAME, "tr")[1:]:  # Ignora o cabeçalho
        cols = [td.text for td in row.find_elements(By.TAG_NAME, "td")]
        data.append(cols)

    # Criar DataFrame e salvar como CSV
    df = pd.DataFrame(data, columns=headers)

    return df


In [7]:
# Exemplo de uso:
# url_strava = "https://www.strava.com/activities/9680684368/laps"
# df_voltas = pega_tabela_voltas()
# print(df_voltas.head())  # Exibe as primeiras linhas do DataFrame

### Pega tabela de dados pagina inicial

In [8]:
def pega_tabela_inicial():
    """
    Extrai a tabela de voltas da página inicial do segmento do atleta do Strava
    """

    # Encontrar a tabela
    table = driver.find_element(By.XPATH, '//*[@id="run-efforts-table"]/section/table')

    # Pegar os cabeçalhos
    headers = [th.text for th in table.find_elements(By.TAG_NAME, "th")]

    # Pegar os dados da tabela
    data = []
    for row in table.find_elements(By.TAG_NAME, "tr")[1:]:  # Ignora o cabeçalho
        cols = [td.text for td in row.find_elements(By.TAG_NAME, "td")]
        data.append(cols)

    # Criar DataFrame e salvar como CSV
    df = pd.DataFrame(data, columns=headers)

    return df


# Exemplo de uso:
# url_strava = "https://www.strava.com/activities/9681673457#3128470765882939836"
# df_voltas = pega_tabela_inicial()
# print(df_voltas.head())  # Exibe as primeiras linhas do DataFrame

### Função navegação

In [9]:
def verifica_nomes_atletas():
    try:
        # Tente encontrar um elemento que contenha os nomes dos atletas
        driver.find_element(By.XPATH, '//*[@id="results"]/table/tbody/tr/td[2]/a')
        return True  # Se encontrar, retorna True (há nomes de atletas)
    except NoSuchElementException:
        return False


def verifica_num_pag():
    try:
        num_paginas = len(
            driver.find_elements(By.XPATH, '//*[@id="results"]/nav/ul/li')
        )
        if num_paginas == 0:
            num_paginas = 1
        else:
            num_paginas
    except:
        num_paginas = 1

    return num_paginas


def clicar_na_pagina(i):
    try:
        driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="results"]/nav/ul/li[{i + 1}]/a',
        ).click()
        sleep(1)
    except Exception:
        pass


def obter_nome_atleta(i, j):
    try:
        if i == 1 and j == 1:
            return driver.find_element(
                by=By.XPATH,
                value=f'//*[@id="results"]/table/tbody/tr[{j}]/td[2]/strong/a',
            ).text
        else:
            return driver.find_element(
                by=By.XPATH,
                value=f'//*[@id="results"]/table/tbody/tr[{j}]/td[2]/a',
            ).text
    except Exception:
        return None


def pega_nome_atletas():
    # Selecionando os dados dos corredores
    s = 1
    n = 26
    num_pag = verifica_num_pag()
    nomes_atletas = []

    # Ajustar o número de páginas
    if num_pag == 1:
        num_pag += 1
    else:
        num_pag -= 1

    for i in range(1, num_pag):
        clicar_na_pagina(i)
        for j in range(s, n):
            nome = obter_nome_atleta(i, j)
            if nome:
                nomes_atletas.append(nome)
            else:
                break

    return nomes_atletas

In [10]:
def pega_todos_nomes_da_lista_filtrada(driver):
    """
    A partir da visualização atual, percorre todas as páginas de resultados
    e retorna uma lista com todos os nomes de atletas.
    """
    nomes_encontrados = []
    pagina_atual = 1

    while True:
        # Espera a tabela de resultados estar presente
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="results"]/table/tbody/tr')
                )
            )
        except TimeoutException:
            # Se não encontrar tabela, provavelmente não há resultados para este filtro.
            break

        # Coleta os nomes da página atual (usando o XPath flexível que já corrigimos)
        linhas_atletas = driver.find_elements(
            By.XPATH, '//*[@id="results"]/table/tbody/tr'
        )
        for linha in linhas_atletas:
            try:
                nome = linha.find_element(By.XPATH, ".//td[2]//a").text
                if nome:
                    nomes_encontrados.append(nome)
            except NoSuchElementException:
                continue

        # Tenta ir para a próxima página
        pagina_atual += 1
        try:
            # Tenta encontrar o link da próxima página e clicar nele
            link_proxima_pagina = driver.find_element(
                By.XPATH, f'//div[@class="paginator"]//a[text()="{pagina_atual}"]'
            )
            link_proxima_pagina.click()
        except NoSuchElementException:
            # Se não encontrou o link para a próxima página, é porque chegamos ao fim.
            break

    return nomes_encontrados

### Pegando dados atletas

In [11]:
"""def pega_dados_atletas():
    # Selecionando os dados dos corredores
    s = 1
    n = 1
    num_pag = verifica_num_pag()

    for i in range(1, num_pag-1):
        clicar_na_pagina(i)
        for j in range(s, n):
            try: # Verificando em qual aba está - 1 a 5 - caso não esteja na aba certa, clica-se na aba correta
                driver.find_element(
                    by=By.XPATH,
                    value=f'//*[@id="results"]/nav/ul/li[{i+1}]/a',
                    ).click()
                sleep(2)
                driver.find_element(
                    by = By.XPATH,
                    value = f'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a',
                    ).click() # Clicando no link do atleta
                driver.back()
                sleep(2)
        
            except Exception: # Caso ja esteja na aba correta, vai direto pro atleta
                try:
                    driver.find_element(
                        by = By.XPATH,
                        value = f'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a',
                        ).click() # Clicando no link do atleta
                    driver.back()
                    sleep(2)
                except Exception:
                    break"""

'def pega_dados_atletas():\n    # Selecionando os dados dos corredores\n    s = 1\n    n = 1\n    num_pag = verifica_num_pag()\n\n    for i in range(1, num_pag-1):\n        clicar_na_pagina(i)\n        for j in range(s, n):\n            try: # Verificando em qual aba está - 1 a 5 - caso não esteja na aba certa, clica-se na aba correta\n                driver.find_element(\n                    by=By.XPATH,\n                    value=f\'//*[@id="results"]/nav/ul/li[{i+1}]/a\',\n                    ).click()\n                sleep(2)\n                driver.find_element(\n                    by = By.XPATH,\n                    value = f\'//*[@id="results"]/table/tbody/tr[{j}]/td[3]/a\',\n                    ).click() # Clicando no link do atleta\n                driver.back()\n                sleep(2)\n        \n            except Exception: # Caso ja esteja na aba correta, vai direto pro atleta\n                try:\n                    driver.find_element(\n                        by = 

In [12]:
def coletar_dados_de_filtro(
    driver, url_base, xpath_botao_expandir, xpath_lista_opcoes, nome_coluna
):
    """
    Função genérica para coletar dados de um filtro (sexo, idade, peso).

    :param driver: A instância do Selenium WebDriver.
    :param url_base: A URL da página de leaderboard.
    :param xpath_botao_expandir: O XPath do botão que abre o menu de filtros.
    :param xpath_lista_opcoes: O XPath para a lista <ul> que contém as opções de filtro.
    :param nome_coluna: O nome da coluna para o DataFrame final ('sexo', 'faixa_etaria', etc.).
    :return: Um DataFrame com os dados do filtro.
    """
    print(f"\n--- Coletando dados para o filtro: {nome_coluna} ---")
    driver.get(url_base)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath_botao_expandir))
    )

    # 1. Encontra e expande o menu de filtro (APENAS UMA VEZ)
    driver.find_element(By.XPATH, xpath_botao_expandir).click()

    # 2. Pega todos os links de opções de filtro para evitar "Stale Element"
    lista_opcoes = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath_lista_opcoes))
    )
    opcoes = lista_opcoes.find_elements(By.TAG_NAME, "a")
    links_e_textos = [
        (opt.get_attribute("href"), opt.text) for opt in opcoes if opt.text
    ]

    dados_filtro = []
    # 3. Itera sobre os links e textos coletados
    for link, texto_filtro in links_e_textos:
        if not link:
            continue

        print(f"Processando: {texto_filtro}...")
        driver.get(
            link
        )  # Navega diretamente para a URL do filtro, mais rápido e seguro

        # 4. Pega todos os nomes para o filtro atual
        nomes_atletas = pega_todos_nomes_da_lista_filtrada(driver)

        # 5. Adiciona os dados à lista de resultados
        for nome in nomes_atletas:
            dados_filtro.append({nome_coluna: texto_filtro, "nome_atleta": nome})

    return pd.DataFrame(dados_filtro)

In [13]:
## Nova função


def pega_dados_atletas_corrigido(driver, url_base_prova: str, nome_arquivo: str):
    """
    Percorre a lista de atletas de forma robusta, coletando os dados de cada um.

    Esta função navega página por página, coleta os links e nomes dos atletas
    de cada página antes de visitá-los, evitando erros de elementos "stale".
    """
    driver.get(url_base_prova)
    sleep(3)  # Espera inicial para a página carregar

    # 1. Descobrir o número de páginas
    num_paginas = verifica_num_pag()
    # Se a função retorna o número de 'li', e a última é 'next', ajustamos.
    # Se sua função já retorna o número correto, pode simplificar aqui.
    # No seu código original, num_pag era usado de forma um pouco confusa.
    # Vamos simplificar para ir de 1 até o número total de páginas.
    print(f"Total de páginas encontradas: {num_paginas}")

    dados_coletados = []

    # 2. Loop principal para cada PÁGINA de resultados
    for i in range(1, num_paginas + 1):
        # Constrói a URL da página específica. É mais confiável que clicar.
        # A maioria dos sites usa um parâmetro como '?page=X'
        url_pagina_atual = f"{url_base_prova}?page={i}"
        print(f"Acessando a página {i}: {url_pagina_atual}")
        driver.get(url_pagina_atual)
        sleep(3)  # Espera a página de resultados carregar

        # 3. Coletar todos os links e nomes de atletas ANTES de sair da página
        links_atletas = []
        nomes_atletas = []
        try:
            # Encontra todas as linhas (tr) da tabela de resultados
            linhas_tabela = driver.find_elements(
                By.XPATH, '//*[@id="results"]/table/tbody/tr'
            )

            for linha in linhas_tabela:
                try:
                    # Em cada linha, pega o link (href) e o nome do atleta
                    # Em cada linha, pega o link (href) e o nome do atleta de forma mais flexível
                    link_element = linha.find_element(
                        By.XPATH, ".//td[3]//a"
                    )  # Use // para mais robustez
                    nome_element = linha.find_element(
                        By.XPATH, ".//td[2]//a"
                    )  # Use // para encontrar o <a> mesmo dentro de um <strong>

                    links_atletas.append(link_element.get_attribute("href"))
                    nomes_atletas.append(nome_element.text)
                except NoSuchElementException:
                    # Ignora linhas que não são de atletas (cabeçalhos, etc.)
                    continue
        except Exception as e:
            print(f"Erro ao coletar links na página {i}: {e}")
            continue  # Pula para a próxima página

        print(f"Encontrados {len(links_atletas)} atletas na página {i}.")

        # 4. Loop para visitar cada LINK de atleta coletado
        for j, link in enumerate(links_atletas):
            atleta_nome = nomes_atletas[j]
            print(f"  -> Coletando dados de: {atleta_nome}")

            try:
                driver.get(link)
                sleep(2)  # Espera a página do atleta carregar

                df = None
                # Verifica se existe a aba "Voltas" e pega a tabela
                if tem_tabela_voltas():
                    # A função tem_tabela_voltas já clica, então só esperamos e pegamos a tabela
                    sleep(2)
                    df = pega_tabela_voltas()
                else:
                    # Se não tem 'Voltas', pega a tabela inicial
                    sleep(2)
                    df = pega_tabela_inicial()

                if df is not None:
                    # Adiciona o nome do atleta à tabela
                    df.insert(0, "Nome Atleta", atleta_nome)
                    dados_coletados.append(df)
                else:
                    print(f"     [Aviso] Nenhuma tabela encontrada para {atleta_nome}")

            except Exception as e:
                print(
                    f"     [Erro] Não foi possível processar o atleta {atleta_nome}. Erro: {e}"
                )
                # Não precisa usar driver.back(), pois o próximo passo do loop
                # é navegar para a URL do próximo atleta.
                continue

    # 5. Concatenar todos os DataFrames ao final
    if dados_coletados:
        df_final = pd.concat(dados_coletados, ignore_index=True)
        # Salve com um nome de sua escolha
        df_final.to_csv(nome_arquivo, index=False)
        print(f"\nProcesso concluído! Dados salvos em '{nome_arquivo}.csv'")
        return df_final
    else:
        print("\nNenhum dado foi coletado.")
        return None


### Pega 'Sexo' Atletas

In [14]:
def pega_sexo_nomes(df):
    driver.refresh()
    for i in range(2, 4):
        # Clica no expandir os sexos
        driver.find_element(
            by=By.XPATH,
            value='//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/button',
        ).click()
        sleep(2)

        # Clica no sexo
        driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li[{i}]/a',
        ).click()

        # Pega o texto do sexo
        sexo_texto = driver.find_element(
            by=By.XPATH,
            value=f'//*[@id="segment-results"]/div[2]/table/tbody/tr/td[4]/div/ul/li[{i}]/a',
        ).text

        sleep(3)

        # Verifica se há nomes de atletas
        existe_nome = verifica_nomes_atletas()

        if existe_nome:
            # Pega os nomes dos atletas
            nomes_atletas = pega_nome_atletas()
            # Cria um DataFrame temporário com os novos dados
            temp_df = pd.DataFrame(
                {
                    "sexo": [sexo_texto] * len(nomes_atletas),
                    "nome_atleta": nomes_atletas,
                }
            )
            # Concatena o DataFrame temporário ao principal
            df = pd.concat([df, temp_df], ignore_index=True)

    return df

### Pega Faixa Idade Atleta

In [15]:
def pega_faixas_nomes(df):
    driver.refresh()
    for i in range(2, 11):
        # Clica no expandir as idades
        driver.find_element(
            by=By.XPATH, value='//*[@id="premium-enhanced"]/ul/ul[2]/li[11]/button'
        ).click()
        sleep(2)

        # Clica na faixa idade
        driver.find_element(
            by=By.XPATH, value=f'//*[@id="premium-enhanced"]/ul/ul[2]/li[{i}]/a'
        ).click()

        # Pega o texto da faixa etaria
        faixa_texto = driver.find_element(
            by=By.XPATH, value=f'//*[@id="premium-enhanced"]/ul/ul[2]/li[{i}]/a'
        ).text

        sleep(3)

        # Verifica se há nomes de atletas
        existe_nome = verifica_nomes_atletas()

        if existe_nome:
            # Pega os nomes dos atletas
            nomes_atletas = pega_nome_atletas()
            # Cria um DataFrame temporário com os novos dados
            temp_df = pd.DataFrame(
                {
                    "faixa_etaria": [faixa_texto] * len(nomes_atletas),
                    "nome_atleta": nomes_atletas,
                }
            )
            # Concatena o DataFrame temporário ao principal
            df = pd.concat([df, temp_df], ignore_index=True)

    return df

### Pega Peso Atletas

In [16]:
def pega_pesos_nomes(df):
    driver.refresh()
    for i in range(2, 10):
        # Clica no expandir os pesos
        driver.find_element(
            by=By.XPATH, value='//*[@id="premium-enhanced"]/ul/ul[3]/li[10]/button'
        ).click()
        sleep(2)

        # Clica na faixa peso
        driver.find_element(
            by=By.XPATH, value=f'//*[@id="premium-enhanced"]/ul/ul[3]/li[{i}]/a'
        ).click()

        # Pega o texto da faixa etaria
        peso_texto = driver.find_element(
            by=By.XPATH, value=f'//*[@id="premium-enhanced"]/ul/ul[3]/li[{i}]/a'
        ).text

        sleep(3)

        # Verifica se há nomes de atletas
        existe_nome = verifica_nomes_atletas()

        if existe_nome:
            # Pega os nomes dos atletas
            nomes_atletas = pega_nome_atletas()
            # Cria um DataFrame temporário com os novos dados
            temp_df = pd.DataFrame(
                {
                    "peso": [peso_texto] * len(nomes_atletas),
                    "nome_atleta": nomes_atletas,
                }
            )
            # Concatena o DataFrame temporário ao principal
            df = pd.concat([df, temp_df], ignore_index=True)

    return df

## Função Geral

In [17]:
def pega_dados_atletas_corrigido(url_base_prova: str, nome_arquivo: str):
    """
    Percorre a lista de atletas de forma robusta, coletando os dados de cada um.

    Esta função navega página por página, coleta os links e nomes dos atletas
    de cada página antes de visitá-los, evitando erros de elementos "stale".
    """
    driver.get(url_base_prova)
    sleep(3)  # Espera inicial para a página carregar

    # 1. Descobrir o número de páginas
    num_paginas = verifica_num_pag()
    # Se a função retorna o número de 'li', e a última é 'next', ajustamos.
    # Se sua função já retorna o número correto, pode simplificar aqui.
    # No seu código original, num_pag era usado de forma um pouco confusa.
    # Vamos simplificar para ir de 1 até o número total de páginas.
    print(f"Total de páginas encontradas: {num_paginas}")

    dados_coletados = []

    # 2. Loop principal para cada PÁGINA de resultados
    for i in range(1, num_paginas + 1):
        # Constrói a URL da página específica. É mais confiável que clicar.
        # A maioria dos sites usa um parâmetro como '?page=X'
        url_pagina_atual = f"{url_base_prova}?page={i}"
        print(f"Acessando a página {i}: {url_pagina_atual}")
        driver.get(url_pagina_atual)
        # Espera a tabela de resultados da página atual carregar
        try:
            WebDriverWait(driver, 10).until(  # <<< MUDANÇA IMPORTANTE
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="results"]/table/tbody/tr')
                )
            )
        except TimeoutException:
            print(
                f"Aviso: A página {i} não carregou a tabela de resultados a tempo. Pulando..."
            )
            continue

        # 3. Coletar todos os links e nomes de atletas ANTES de sair da página
        links_atletas = []
        nomes_atletas = []
        try:
            # Encontra todas as linhas (tr) da tabela de resultados
            linhas_tabela = driver.find_elements(
                By.XPATH, '//*[@id="results"]/table/tbody/tr'
            )

            for linha in linhas_tabela:
                try:
                    # Em cada linha, pega o link (href) e o nome do atleta
                    # Em cada linha, pega o link (href) e o nome do atleta de forma mais flexível
                    link_element = linha.find_element(
                        By.XPATH, ".//td[3]//a"
                    )  # Use // para mais robustez
                    nome_element = linha.find_element(
                        By.XPATH, ".//td[2]//a"
                    )  # Use // para encontrar o <a> mesmo dentro de um <strong>

                    links_atletas.append(link_element.get_attribute("href"))
                    nomes_atletas.append(nome_element.text)
                except NoSuchElementException:
                    # Ignora linhas que não são de atletas (cabeçalhos, etc.)
                    continue
        except Exception as e:
            print(f"Erro ao coletar links na página {i}: {e}")
            continue  # Pula para a próxima página

        print(f"Encontrados {len(links_atletas)} atletas na página {i}.")

        # 4. Loop para visitar cada LINK de atleta coletado
        for j, link in enumerate(links_atletas):
            atleta_nome = nomes_atletas[j]
            print(f"  -> Coletando dados de: {atleta_nome}")

            try:
                driver.get(link)
                sleep(2)  # Espera a página do atleta carregar

                df = None
                # Verifica se existe a aba "Voltas" e pega a tabela
                if tem_tabela_voltas():
                    # A função tem_tabela_voltas já clica, então só esperamos e pegamos a tabela
                    sleep(2)
                    df = pega_tabela_voltas()
                else:
                    # Se não tem 'Voltas', pega a tabela inicial
                    sleep(2)
                    df = pega_tabela_inicial()

                if df is not None:
                    # Adiciona o nome do atleta à tabela
                    df.insert(0, "Nome Atleta", atleta_nome)
                    dados_coletados.append(df)
                else:
                    print(f"     [Aviso] Nenhuma tabela encontrada para {atleta_nome}")

            except Exception as e:
                print(
                    f"     [Erro] Não foi possível processar o atleta {atleta_nome}. Erro:"
                )
                # Não precisa usar driver.back(), pois o próximo passo do loop
                # é navegar para a URL do próximo atleta.
                continue

    # 5. Concatenar todos os DataFrames ao final
    if dados_coletados:
        df_final = pd.concat(dados_coletados, ignore_index=True)
        # Salve com um nome de sua escolha
        df_final.to_csv(nome_arquivo, index=False)
        print("\nProcesso concluído! Dados salvos em 'dados_strava_final.csv'")
        return df_final
    else:
        print("\nNenhum dado foi coletado.")
        return None

In [18]:
def gerar_dataframe_final_completo(url_base_prova: str, nome_arquivo: str):
    """
    Orquestra toda a coleta de dados e os une da forma correta,
    criando uma tabela de atributos antes de fazer o join final.
    """
    nome_arquivo_saida = nome_arquivo  # Remove a extensão do arquivo
    # --- PASSO 1: Coletar os dados principais de tempo ---
    print("--- Iniciando coleta de dados de performance dos atletas ---")
    df_principal = pega_dados_atletas_corrigido(
        url_base_prova=url_base_prova, nome_arquivo=nome_arquivo_saida
    )
    if df_principal is None:
        print("Não foi possível coletar os dados principais. Abortando.")
        return None
    print("--- Coleta de dados de performance concluída ---")

    # --- PASSO 2: Coletar dados de cada filtro separadamente ---
    # Pega os dados de sexo, idade e peso
    df_sexo = pd.DataFrame(columns=["sexo", "nome_atleta"])
    df_sexo_atletas = pega_sexo_nomes(df_sexo)
    ## Pega as faixas etárias e nomes
    df_faixas_idades = pd.DataFrame(columns=["faixa_etaria", "nome_atleta"])
    df_faixas_idades = pega_faixas_nomes(df_faixas_idades)
    ### Coletando dados pesos
    df_pesos = pd.DataFrame(columns=["peso", "nome_atleta"])
    df_pesos = pega_pesos_nomes(df_pesos)

    # --- PASSO 3: Fazer o merge final e único ---
    print("\n--- Unindo dados de performance com a tabela de atributos ---")
    df_final_principal_sexo = pd.merge(
        df_principal,
        df_sexo_atletas,
        how="left",
        left_on="Nome Atleta",
        right_on="nome_atleta",
    )

    # Passo 3.1: Juntar o resultado anterior com df_faixa_idade
    df_final_idades = pd.merge(
        df_final_principal_sexo,
        df_faixas_idades,
        how="left",
        left_on="Nome Atleta",
        right_on="nome_atleta",
    )

    # Passo 3.2: Juntar o resultado anterior com df_pesos
    df_final_pesos = pd.merge(
        df_final_idades,
        df_pesos,
        how="left",
        left_on="Nome Atleta",
        right_on="nome_atleta",
    )

    # Passo 4 (Opcional, mas recomendado): Limpeza
    # O merge mantém as colunas de chave de ambas as tabelas. Vamos remover as colunas
    # 'nome_atleta' duplicadas, que terão sufixos _x e _y.
    # Uma forma mais simples é identificar as colunas 'nome_atleta' e removê-las.
    colunas_para_remover = [
        col for col in df_final_pesos.columns if "nome_atleta" in str(col)
    ]
    df_final = df_final_pesos.drop(columns=colunas_para_remover)

    # --- PASSO 4: Salvar e retornar ---
    nome_final_arquivo = f"final_completo_{nome_arquivo_saida}"
    df_final.to_csv(nome_final_arquivo, index=False, encoding="utf-8-sig")
    print(f"\nPROCESSO CONCLUÍDO! DataFrame final salvo em '{nome_final_arquivo}'")

    return df_final


# --- Exemplo de como chamar a função principal ---
#
# from selenium import webdriver
#
# # Configuração do driver
# driver = webdriver.Chrome()
#
# url_da_prova = "URL_DO_LEADERBOARD_DO_STRAVA"
# nome_base_arquivo = "dados_prova_x.csv"
#
# df_completo = gerar_dataframe_final_completo(driver, url_da_prova, nome_base_arquivo)
#
# if df_completo is not None:
#    print("\nAmostra do DataFrame final:")
#    print(df_completo.head().to_string())
#
# driver.quit()


In [ ]:
# df = pega_dados_atletas_corrigido(
#    url_base_prova="https://www.strava.com/segments/35231066",
#    nome_arquivo="dados_TESTE_1007.csv",
# )
# if df is not None:
#    print(df.head())

In [20]:
# --- Exemplo de como chamar ---
url_da_prova = "https://www.strava.com/segments/32632496"
driver.get(url_da_prova)
df_completo = gerar_dataframe_final_completo(
    url_base_prova=url_da_prova, nome_arquivo="dados_2022_P1.csv"
)
if df_completo is not None:
    print(df_completo.head().to_string())

--- Iniciando coleta de dados de performance dos atletas ---
Total de páginas encontradas: 9
Acessando a página 1: https://www.strava.com/segments/32632496?page=1
Encontrados 25 atletas na página 1.
  -> Coletando dados de: Ayslan Miragaia 🦅
  -> Coletando dados de: Alexandre Santiago
  -> Coletando dados de: Renan Ribeiro(Aru)
  -> Coletando dados de: Cleito Maraga
  -> Coletando dados de: Victor Resende
  -> Coletando dados de: Ezequiel Morales
  -> Coletando dados de: Nonato Mota 🇧🇷
  -> Coletando dados de: 👑 VIRGINIO🏔
  -> Coletando dados de: Natã Valeriano
  -> Coletando dados de: Mikael Martins
  -> Coletando dados de: Breno Cordeiro
  -> Coletando dados de: Amaro Almeida Neto
     [Erro] Não foi possível processar o atleta Amaro Almeida Neto. Erro:
  -> Coletando dados de: Gustavo Henrique
     [Erro] Não foi possível processar o atleta Gustavo Henrique. Erro:
  -> Coletando dados de: Arnaldo Junio Rocha Pinho
  -> Coletando dados de: Péricles Maia / Base TRAIL Run
  -> Coletand

In [28]:
df_lido = pd.read_csv("dados_strava_final.csv")
df_lido.head()

,Nome Atleta,KM,Ritmo,RAI,Elev.,Volta,Distância,Tempo,FC,Cadência
0,Alexandre Santiago,1,4:43 /km,4:40 /km,-1 m,NaN,NaN,NaN,NaN,NaN
1,Alexandre Santiago,2,5:07 /km,3:41 /km,84 m,NaN,NaN,NaN,NaN,NaN
2,Alexandre Santiago,3,5:14 /km,4:12 /km,61 m,NaN,NaN,NaN,NaN,NaN
3,Alexandre Santiago,4,6:30 /km,4:14 /km,94 m,NaN,NaN,NaN,NaN,NaN
4,Alexandre Santiago,5,6:38 /km,4:46 /km,0 m,NaN,NaN,NaN,NaN,NaN


In [29]:
df_lido["Nome Atleta"].nunique()


92

## Juntando os DataFrames

## Segments

In [ ]:
url_1_2022 = "https://www.strava.com/segments/32632496"
url_2_2022 = "https://www.strava.com/segments/32632505"

In [ ]:
df_2022_1 = pega_dados_atletas_corrigido(
    url_base_prova=url_1_2022, nome_arquivo="dados_atletas_2022_1.csv"
)
df_2022_1.head()

### 2022 2